In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

C:\Users\Ratik\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras

Using TensorFlow backend.


In [3]:
(images_train,cls_train),(images_test,cls_test)=keras.datasets.cifar10.load_data()

In [4]:
images_train.shape,cls_train.shape

((50000, 32, 32, 3), (50000, 1))

In [5]:
labels_train=[]
for i in cls_train:
    classes=np.zeros(10)
    classes[i]=1
    labels_train.append(classes)
labels_train=np.array(labels_train)
labels_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [6]:
labels_test=[]
for i in cls_test:
    classes=np.zeros(10)
    classes[i]=1
    labels_test.append(classes)
labels_test=np.array(labels_test)
labels_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [7]:
# import cifar10

In [8]:
# images_train, cls_train, labels_train = cifar10.load_training_data()
# images_test, cls_test, labels_test = cifar10.load_test_data()

In [9]:
images_train.shape

(50000, 32, 32, 3)

In [10]:
32*32*3

3072

In [11]:
n_input=3072
input_width=32
input_height=32
input_channels=3

n_conv1=32
n_conv2=64
conv1_k=5
conv2_k=5

n_hidden=1024
n_classes=10

pooling_ws=2


In [12]:
weights={
    
    "c1":tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "c2":tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),

    "d1":tf.Variable(tf.random_normal([8*8*64,n_hidden])),
    "out":tf.Variable(tf.random_normal([n_hidden,n_classes]))
       
}

biases={
    
    "c1":tf.Variable(tf.random_normal([n_conv1])),
    "c2":tf.Variable(tf.random_normal([n_conv2])),
    "d1":tf.Variable(tf.random_normal([n_hidden])),
    "out":tf.Variable(tf.random_normal([n_classes]))
    
    
}

In [13]:
images_train.shape

(50000, 32, 32, 3)

In [14]:
conv2_out_width = input_width//(pooling_ws*pooling_ws)
conv2_out_height = input_height//(pooling_ws*pooling_ws) 
dense_input = conv2_out_width*conv2_out_height * n_conv2

In [15]:
def conv(x,weights,bias,stride=1):
    #x=tf.reshape(x,shape=[50000,32,32,3])
    conv_out = tf.nn.conv2d(x, weights, padding="SAME", strides=[1,stride,stride,1])
    
    out = tf.nn.bias_add(conv_out, bias)
    out = tf.nn.relu(out)

    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding="SAME", ksize=[1, k, k, 1], strides = [1, k, k, 1])
    
    

In [16]:
def cnn(x,weights,biases):

    #x = tf.reshape(x, shape=[-1,input_width, input_height, input_channels ])

    print(x.shape)
    conv1=conv(x,weights["c1"],biases["c1"])
    print(conv1.shape)
    conv1=maxpooling(conv1,k=pooling_ws)
    
    conv2=conv(conv1,weights["c2"],biases["c2"])
    print(conv2.shape)
    conv2=maxpooling(conv2,k=pooling_ws)
    
    hidden_input=tf.reshape(conv2,shape=[-1,dense_input])
    hidden_output = tf.add(tf.matmul(hidden_input, weights['d1']), biases['d1'])
    hidden_output = tf.nn.relu(hidden_output)
    
    print(hidden_output.shape)
    
    in_output_layer = tf.add(tf.matmul(hidden_output, weights['out']), biases['out'])
    output = in_output_layer
    print(output.shape)
    return output
    

In [17]:
x = tf.placeholder("float", [None, 32,32,3])
y = tf.placeholder(tf.int32, [None,10])

In [18]:
pred=cnn(x,weights,biases)

(?, 32, 32, 3)
(?, 32, 32, 32)
(?, 16, 16, 64)
(?, 1024)
(?, 10)


In [19]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [20]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize_step = optimizer.minimize(cost)

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [23]:
# for i in range(25):
c, _ = sess.run([cost, optimize_step], feed_dict={x:images_train[:10], y:labels_train[:10]})

In [24]:
images_train.shape

(50000, 32, 32, 3)

In [25]:
predictions = tf.argmax(pred, axis = 1)
actual_labels = tf.argmax(y, axis = 1)
correct_predictions = tf.equal(predictions,actual_labels)

correct_preds = sess.run(correct_predictions, feed_dict={x:images_train[:100], y:labels_train[:100]} )
a=correct_preds.sum()
print(a)

14


In [26]:
predictions = tf.argmax(pred, axis = 1)
actual_labels = tf.argmax(y, axis = 1)
correct_predictions = tf.equal(predictions,actual_labels)

correct_preds = sess.run(correct_predictions, feed_dict={x:images_test[:100], y:labels_test[:100]} )
a=correct_preds.sum()
print(a)

5


In [27]:
228/1000

0.228

In [28]:
import numpy as np

def next_batch(data, labels,num):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# Xtr, Ytr = np.arange(0, 10), np.arange(0, 100).reshape(10, 10)
# print(Xtr)
# print(Ytr)

# Xtr, Ytr = next_batch(5, Xtr, Ytr)
# print('\n5 random samples')
# print(Xtr)
# print(Ytr)

In [29]:
def nextbatch(x_train,y_train,batch_size,j):
    batch_x=np.array(x_train[j*batch_size:j*batch_size+batch_size])
    batch_y=np.array(y_train[j*batch_size:j*batch_size+11 ])
    
    return batch_x,batch_y
    

In [30]:
batch_size = 100
for i in range(25):
    num_batches = int(50000/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = next_batch(images_train,labels_train,batch_size) 
        c, _ = sess.run([cost, optimize_step], feed_dict={x:batch_x, y:batch_y})
        total_cost += c
        
    print(total_cost)
    break

677793115.5957241


In [31]:
b_x, b_y = next_batch(images_train,cls_train,100)

In [32]:
images_train.shape,b_x.shape

((50000, 32, 32, 3), (100, 32, 32, 3))

In [33]:
cls_train.shape,b_y.shape

((50000, 1), (100, 1))